In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime


historical_data = pd.read_csv('data.csv')

historical_data['Expiry'] = pd.to_datetime(historical_data['Expiry'])
historical_data['Date'] = pd.to_datetime(historical_data['Date'])
historical_data['days_to_expiry'] = (historical_data['Expiry'] - historical_data['Date']).dt.days




<ipython-input-38-8b60227b9c2b>:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  historical_data['Expiry'] = pd.to_datetime(historical_data['Expiry'])
<ipython-input-38-8b60227b9c2b>:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  historical_data['Date'] = pd.to_datetime(historical_data['Date'])


In [39]:
# Feature engineering
import tensorflow as tf
#print(historical_data.columns)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features_to_scale = ['index_Open', 'index_High', 'index_Low', 'index_Close', 'index_Gap']
X_to_scale = historical_data[features_to_scale].values
X_to_keep = historical_data['days_to_expiry'].values

scaler_X = MinMaxScaler(feature_range=(0, 1))
print(X_to_scale.shape)
X_scaled = scaler_X.fit_transform(X_to_scale)

X_final = np.hstack((X_scaled, X_to_keep.reshape(-1, 1)))
#print(X_final)
y = historical_data[['straddle_Open', 'straddle_High', 'straddle_Low', 'straddle_Close']].values

scaler_y = MinMaxScaler(feature_range=(0, 1))

y_scaled = scaler_y.fit_transform(y)

# Reshape input data for LSTM
X_final = X_final.reshape((X_final.shape[0], 1, X_final.shape[1]))

# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X_final, y_scaled, test_size=0.2, random_state=42)

# Model building
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=4))  # 4 outputs for Straddle Open, High, Low, Close

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer= optimizer, loss='mse')

# Model training
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test , y_test) , callbacks=(early_stopping))


predictions = model.predict(X_test)
predictions_unscaled = scaler_y.inverse_transform(predictions)
print(predictions_unscaled)
best_val_loss = min(history.history['val_loss'])
print("Best Validation Loss:", best_val_loss)
best_loss = min(history.history['loss'])
print("Best Training Loss:", best_loss)



(30, 5)
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.6323 - val_loss: 0.5259
Epoch 2/100
1/1 [==============================] - 0s 39ms/step - loss: 0.6003 - val_loss: 0.5075
Epoch 3/100
1/1 [==============================] - 0s 40ms/step - loss: 0.5712 - val_loss: 0.4908
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 0.5443 - val_loss: 0.4753
Epoch 5/100
1/1 [==============================] - 0s 44ms/step - loss: 0.5189 - val_loss: 0.4605
Epoch 6/100
1/1 [==============================] - 0s 39ms/step - loss: 0.4948 - val_loss: 0.4462
Epoch 7/100
1/1 [==============================] - 0s 40ms/step - loss: 0.4716 - val_loss: 0.4323
Epoch 8/100
1/1 [==============================] - 0s 51ms/step - loss: 0.4492 - val_loss: 0.4187
Epoch 9/100
1/1 [==============================] - 0s 39ms/step - loss: 0.4275 - val_loss: 0.4053
Epoch 10/100
1/1 [==============================] - 0s 40ms/step - loss: 0.4065 - val_loss: 0.3920
Epoch 11/100


In [40]:
today_date = datetime.strptime('2024-03-11', '%Y-%m-%d')
# Calculate DTE for the next day
next_day_dte = 2

latest_index_data = historical_data[historical_data['Date'] == '11/03/2024']
#print(latest_index_data)
latest_index_high = latest_index_data['index_High'].values
latest_index_low = latest_index_data['index_Low'].values
latest_index_close = latest_index_data['index_Close'].values
latest_index_open = latest_index_data['index_Open'].values
latest_index_gap = latest_index_data['index_Gap'].values


latest_index_open = np.array(latest_index_open).reshape(-1, 1)
#print(latest_index_open.shape)  # has hape (0,1)
latest_index_high = np.array(latest_index_high).reshape(-1, 1)
latest_index_low = np.array(latest_index_low).reshape(-1, 1)
latest_index_close = np.array(latest_index_close).reshape(-1, 1)
latest_index_gap = np.array(latest_index_gap).reshape(-1, 1)

# #next day features should have shape of 1,5
# next_day_features = np.array([latest_index_open, latest_index_high, latest_index_low, latest_index_close , latest_index_gap])
# print(next_day_features.shape)
next_day_features = np.hstack((latest_index_open, latest_index_high, latest_index_low, latest_index_close, latest_index_gap))
print(next_day_features.shape)  # Should print (1, 5)

next_day_features_scaled = scaler_X.fit_transform(next_day_features)

array_1x1 = np.array([[2]])
next_day_features_scaled = np.hstack((next_day_features_scaled, array_1x1))
# Reshape input data for LSTM
next_day_features_scaled = next_day_features_scaled.reshape((next_day_features_scaled.shape[0], 1, next_day_features_scaled.shape[1]))
# Predict Straddle Open, High, Low, Close for the next day
next_day_predictions_scaled = model.predict(next_day_features_scaled)
next_day_predictions_unscaled = scaler_y.inverse_transform(next_day_predictions_scaled)
print('Predicted Straddle Open, High, Low, Close for the next day for 2dte:', next_day_predictions_unscaled)


(1, 5)
1/1 [==============================] - 0s 20ms/step
Predicted Straddle Open, High, Low, Close for the next day: [[170.34102  171.92511  122.06615  102.704414]]
